# Summary

Try fuzzy matching mis-transcribed names. 

Maybe also try to handle this better for a new contact - e.g. search google/wikipedia to find who I *meant* rather than who Alexa *heard* me say.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from fuzzywuzzy import fuzz, process
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import unidecode
import wikipedia as wiki

from jabberwocky.config import C
from jabberwocky.external_data import wiki_page, wiki_data
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    ConversationManager, PromptManager, GPT
from htools import *

In [8]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


# TODO

GOAL

- avoid saving names with my typos (e.g. Apollo ohno (should have 1 L))
- but avoid saving wiki names with parentheses (e.g. (john smith (american wrestler)")
- and avoid saving wiki names with weird spacing (e.g. "J. K. Rowling")
- and avoid saving incorrect wiki names (e.g. I search for Lex Fridman but get Lex Frieden)

In [73]:
name = 'J. K. Rowling'
n = name.count('. ')
re.sub('. ', '.', name, count=n - 1)

'J.K. Rowling'

In [108]:
def cleanup_wiki_name(name):
    """
    'J. K. Rowling' -> 'J.K. Rowling'
    'A. B. C. Lastname' -> 'A.B.C. Lastname'
    'Ted P. Chang' -> 'Ted P. Chang'
    'John Smith (American Wrestler)' - > 'John Smith'
    """
    n = name.count('. ')
    if n > 1:
        name = re.sub('. ', '.', name, count=n - 1)
    name = name.split('(')[0].strip()
    return unidecode.unidecode(name)

In [98]:
def namecase(name):
    """Similar to titlecase but don't force lowercase letters after the first
    one in each word:
    
    name = 'Yann LeCun'
    name.title()   # 'Yann Lecun'
    namecase(name) # 'Yann LeCun'
    """
    chunks = name.split()
    return ' '.join(chunk[0].upper() + chunk[1:] for chunk in chunks)

In [109]:
cleanup_wiki_name('J. K. Rowling')

'J.K. Rowling'

In [110]:
cleanup_wiki_name('A. B. C. Lastname')

'A.B.C. Lastname'

In [111]:
cleanup_wiki_name('Ted P. Chang')

'Ted P. Chang'

In [112]:
cleanup_wiki_name('John Smith (American Wrestler)')

'John Smith'

In [113]:
cleanup_wiki_name('Beyoncé')

'Beyonce'

In [150]:
conv = ConversationManager()

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [151]:
conv.personas()

['Albert Einstein',
 'Alex Honnold',
 'Andrej Karpathy',
 'Apolo Ohno',
 'Archimedes',
 'Barack Obama',
 'Beyonce',
 'Brandon Sanderson',
 'Donald Glover',
 'Douglas Adams',
 'Dr Seuss',
 'Elijah Wood',
 'Ezra Klein',
 'George Harrison',
 'George Orwell',
 'God',
 'Grace Hopper',
 'Jk Rowling',
 'Kanye West',
 'Lex Fridman',
 'Marie Curie',
 'Mark Zuckerberg',
 'Maya Angelou',
 'Michelle Obama',
 'Neil Degrasse Tyson',
 'Nicolas Cage',
 'Robert Sapolsky',
 'Shel Silverstein',
 'Simone Biles',
 'Sylvia Plath',
 'Yann Lecun']

In [ ]:
'Andrej Karpathy'
'Yann Lecun'

In [134]:
wrong_names = [
    # Alexa mis-transcriptions.
    "Andre Carter",
    'Beyoncé',
    "Lex Frieden",
    "Neil De Grasse Tyson",
    "jan lechon",
    # Names that are actually different. Want to avoid false positives.
    "Annie Klein",
    "Ezra Miller",
    "Brendan Schaub",
    "JK Simmons",
    "Simon Blair",
    # Names that are not identical but should still find matches.
    "Nic Cage",
    "Rob Sapolsky",
    "Kanye",
    "Don Glover"
]

In [156]:
sorted(
    [(name, conv.nearest_persona(name)) for name in wrong_names],
    key=lambda x: x[1][1]
)

[('Brendan Schaub', ('Brandon Sanderson', 0.52)),
 ('JK Simmons', ('Simone Biles', 0.55)),
 ('Andre Carter', ('Andrej Karpathy', 0.67)),
 ('Annie Klein', ('Ezra Klein', 0.67)),
 ('Ezra Miller', ('Ezra Klein', 0.67)),
 ('jan lechon', ('Yann Lecun', 0.7)),
 ('Simon Blair', ('Simone Biles', 0.7)),
 ('Lex Frieden', ('Lex Fridman', 0.82)),
 ('Nic Cage', ('Nicolas Cage', 0.86)),
 ('Don Glover', ('Donald Glover', 0.87)),
 ('Rob Sapolsky', ('Robert Sapolsky', 0.89)),
 ('Kanye', ('Kanye West', 0.9)),
 ('Neil De Grasse Tyson', ('Neil Degrasse Tyson', 0.97)),
 ('Beyoncé', ('Beyonce', 1.0))]

In [155]:
sorted(
    [(conv.process_name(name), conv.nearest_persona(conv.process_name(name)))
     for name in wrong_names],
    key=lambda x: x[1][1]
)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


[('brendan_schaub', ('brandon_sanderson', 0.52)),
 ('jk_simmons', ('simone_biles', 0.55)),
 ('andre_carter', ('andrej_karpathy', 0.67)),
 ('annie_klein', ('ezra_klein', 0.67)),
 ('ezra_miller', ('ezra_klein', 0.67)),
 ('jan_lechon', ('yann_lecun', 0.7)),
 ('simon_blair', ('simone_biles', 0.7)),
 ('nic_cage', ('nicolas_cage', 0.8)),
 ('lex_frieden', ('lex_fridman', 0.82)),
 ('don_glover', ('donald_glover', 0.87)),
 ('rob_sapolsky', ('robert_sapolsky', 0.89)),
 ('kanye', ('Kanye West', 0.9)),
 ('neil_de_grasse_tyson', ('neil_degrasse_tyson', 0.97)),
 ('beyoncé', ('Beyonce', 1.0))]

In [19]:
page = wiki_page('apollo ohno')
page

<WikipediaPage 'Apolo Ohno'>

In [46]:
name2page = {}
for name in tqdm(conv.personas()):
    args = ['computer science'] if 'lecun' in name.lower() else []
    name2page[name] = wiki_page(name, *args)

  0%|          | 0/31 [00:00<?, ?it/s]

/Users/hmamin/jabberwocky/lib/jabberwocky/external_data.py:292: UserWarning: Page not found. Trying to auto-select correct match.
  if '(disambiguation)' in match: continue


In [103]:
{name: (page.title, cleanup_wiki_name(page.title), name == page.title)
 for name, page in name2page.items()}

{'Albert Einstein': ('Albert Einstein', 'Albert Einstein', True),
 'Alex Honnold': ('Alex Honnold', 'Alex Honnold', True),
 'Andrej Karpathy': ('Andrej Karpathy', 'Andrej Karpathy', True),
 'Apollo Ohno': ('Apolo Ohno', 'Apolo Ohno', False),
 'Archimedes': ('Archimedes', 'Archimedes', True),
 'Barack Obama': ('Barack Obama', 'Barack Obama', True),
 'Beyonce': ('Beyoncé', 'Beyoncé', False),
 'Brandon Sanderson': ('Brandon Sanderson', 'Brandon Sanderson', True),
 'Donald Glover': ('Donald Glover', 'Donald Glover', True),
 'Douglas Adams': ('Douglas Adams', 'Douglas Adams', True),
 'Dr Seuss': ('Dr. Seuss', 'Dr. Seuss', False),
 'Elijah Wood': ('Elijah Wood', 'Elijah Wood', True),
 'Ezra Klein': ('Ezra Klein', 'Ezra Klein', True),
 'George Harrison': ('George Harrison', 'George Harrison', True),
 'George Orwell': ('George Orwell', 'George Orwell', True),
 'God': ('God', 'God', True),
 'Grace Hopper': ('Grace Hopper', 'Grace Hopper', True),
 'Jk Rowling': ('J. K. Rowling', 'J.K. Rowling', 

In [65]:
wiki_page("JT Snow")

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


<WikipediaPage 'J. T. Snow'>

In [64]:
conv.process_name('J. K. Rowling')

'j_k_rowling'

In [54]:
conv.process_name('yann lecun', True)

'Yann Lecun'

In [55]:
wiki_page('john smith', debug=True)

/Users/hmamin/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/hmamin/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "John Smith" may refer to: 
John Smith (anatomist and chemist)
John Blair Smith
John Smith (Cambridge, 1766)
John Smith (astronomer)
John Smith (lexicographer)
John Augustine Smith
John Smith (botanist)
John Smith (physician)
John Alexander Smith (physician)
J. Lawrence Smith
John Smith (dentist)
John Campbell Smith
John Donnell Smith
John McGarvie Smith
John Bernhardt Smith
John Alexander Smith
John Maynard Smith
John Cyril Smith
John Derek Smith
John Smith (sociologist)
John D. Smith
John H. Smith (mathematician)
John E. Sharwood Smith
John Edwin Smith
John Merlin Powis Smith
John Smith (engraver)
John Smith (English poet)
John Christopher Smith
John Warwick Smith
John Stafford Smith
John Raphael Smith
John Thomas Smith (engraver)
John Smith (clockmaker)
John Rubens Smith
John Smith (architect)
John Smith (art historian)
John Orrin Smith
John Frederick Smith
John Rowson Smith
John Kelday Smith
John G. Smith (poet)
John Moyr Smith
John Berryman
John Smith (Canadia

In [56]:
wiki_page('john smith', 'wrestler')

DisambiguationError: "John Smith" may refer to: 
John Smith (anatomist and chemist)
John Blair Smith
John Smith (Cambridge, 1766)
John Smith (astronomer)
John Smith (lexicographer)
John Augustine Smith
John Smith (botanist)
John Smith (physician)
John Alexander Smith (physician)
J. Lawrence Smith
John Smith (dentist)
John Campbell Smith
John Donnell Smith
John McGarvie Smith
John Bernhardt Smith
John Alexander Smith
John Maynard Smith
John Cyril Smith
John Derek Smith
John Smith (sociologist)
John D. Smith
John H. Smith (mathematician)
John E. Sharwood Smith
John Edwin Smith
John Merlin Powis Smith
John Smith (engraver)
John Smith (English poet)
John Christopher Smith
John Warwick Smith
John Stafford Smith
John Raphael Smith
John Thomas Smith (engraver)
John Smith (clockmaker)
John Rubens Smith
John Smith (architect)
John Smith (art historian)
John Orrin Smith
John Frederick Smith
John Rowson Smith
John Kelday Smith
John G. Smith (poet)
John Moyr Smith
John Berryman
John Smith (Canadia

In [63]:
wiki.search('John Smith (American Wrestler)')

['John Smith (American wrestler)',
 'John Smith',
 'Davey Boy Smith',
 'John W. Smith',
 'Pat Smith (wrestler)',
 'J.T. Smith (wrestler)',
 'John Layfield',
 'John Nord',
 'John Cena filmography',
 'John Laurinaitis']

In [57]:
wiki.search('john smith wrestler')

['John Smith (American wrestler)',
 'John Smith',
 'Davey Boy Smith',
 'Johnny Smith (wrestler)',
 'J.T. Smith (wrestler)',
 'Davey Boy Smith Jr.',
 'Pat Smith (wrestler)',
 'John Smith (Zambian wrestler)',
 'Freestyle wrestling',
 'Hart wrestling family']

## Test Banana Backend

In [14]:
import requests
import time

from htools import *

In [15]:
GPT.switch('banana')

Switching openai backend to "banana".


In [20]:
pm = PromptManager(verbose=False)

In [21]:
text = 'I wonder if that you give birth to a whole new at whole new realm of fake are instead of mimicking a famous famous artist people cell AI generated art that present is if it was human made'

In [26]:
times = []
results = []
for _ in tqdm(range(10)):
    with block_timer() as bt:
        res = pm.query(task='punctuate_alexa', text=text, 
                       max_tokens=2*len(text.split()))
    print(res[0])
    results.append(res)
    times.append(bt)

  0%|          | 0/10 [00:00<?, ?it/s]

{'engine': 1, 'temperature': 0.2, 'stop': ['Raw transcription:', 'Punctuated transcription:', 'Add any missing punctuation and fix any small'], 'max_tokens': 76, 'prompt': "Add any missing punctuation and fix any small grammatical errors in these raw transcriptions.\n\nRaw transcription: what are your plans for this afternoon\nPunctuated transcription: What are your plans for this afternoon?\n\nRaw transcription: hi john how are you its been a long time since we talked\nPunctuated transcription: Hi John! How are you? It's been a long time since we talked.\n\nRaw transcription: can you elaborate I'm not sure I understand\nPunctuated transcription: Can you elaborate? I'm not sure I understand.\n\nRaw transcription: i want to write a short science fiction novel can you suggest a prompts thank you\nPunctuated transcription: I want to write a short science fiction novel. Can you suggest a prompt? Thank you.\n\nRaw transcription: I wonder if that you give birth to a whole new at whole new re

[TIMER]: Block executed in 5.527 seconds.
['I wonder if that you give birth to a whole new at whole new realm of fake are instead of mimicking a famous famous artist people cell AI generated art that present is if it was human made.']
{'engine': 1, 'temperature': 0.2, 'stop': ['Raw transcription:', 'Punctuated transcription:', 'Add any missing punctuation and fix any small'], 'max_tokens': 76, 'prompt': "Add any missing punctuation and fix any small grammatical errors in these raw transcriptions.\n\nRaw transcription: what are your plans for this afternoon\nPunctuated transcription: What are your plans for this afternoon?\n\nRaw transcription: hi john how are you its been a long time since we talked\nPunctuated transcription: Hi John! How are you? It's been a long time since we talked.\n\nRaw transcription: can you elaborate I'm not sure I understand\nPunctuated transcription: Can you elaborate? I'm not sure I understand.\n\nRaw transcription: i want to write a short science fiction no

In [28]:
times = pd.DataFrame(times)
times.describe()

,duration
count,10.000000
mean,5.137857
std,0.189312
min,4.940122
25%,4.953435
50%,5.180961
75%,5.220249
max,5.526596


In [30]:
GPT.switch('openai')
times2 = []
results2 = []
for _ in tqdm(range(10)):
    with block_timer() as bt:
        res = pm.query(task='punctuate_alexa', text=text, 
                       max_tokens=2*len(text.split()))
    print(res[0])
    results2.append(res)
    times2.append(bt)

Switching openai backend to "openai".


  0%|          | 0/10 [00:00<?, ?it/s]

{'engine': 1, 'temperature': 0.2, 'stop': ['Raw transcription:', 'Punctuated transcription:', 'Add any missing punctuation and fix any small'], 'max_tokens': 76, 'prompt': "Add any missing punctuation and fix any small grammatical errors in these raw transcriptions.\n\nRaw transcription: what are your plans for this afternoon\nPunctuated transcription: What are your plans for this afternoon?\n\nRaw transcription: hi john how are you its been a long time since we talked\nPunctuated transcription: Hi John! How are you? It's been a long time since we talked.\n\nRaw transcription: can you elaborate I'm not sure I understand\nPunctuated transcription: Can you elaborate? I'm not sure I understand.\n\nRaw transcription: i want to write a short science fiction novel can you suggest a prompts thank you\nPunctuated transcription: I want to write a short science fiction novel. Can you suggest a prompt? Thank you.\n\nRaw transcription: I wonder if that you give birth to a whole new at whole new re

[TIMER]: Block executed in 0.614 seconds.
["I wonder if that you give birth to a whole new realm of fake art instead of mimicking a famous famous artist? People's cells generate art that present is if it was human made."]
{'engine': 1, 'temperature': 0.2, 'stop': ['Raw transcription:', 'Punctuated transcription:', 'Add any missing punctuation and fix any small'], 'max_tokens': 76, 'prompt': "Add any missing punctuation and fix any small grammatical errors in these raw transcriptions.\n\nRaw transcription: what are your plans for this afternoon\nPunctuated transcription: What are your plans for this afternoon?\n\nRaw transcription: hi john how are you its been a long time since we talked\nPunctuated transcription: Hi John! How are you? It's been a long time since we talked.\n\nRaw transcription: can you elaborate I'm not sure I understand\nPunctuated transcription: Can you elaborate? I'm not sure I understand.\n\nRaw transcription: i want to write a short science fiction novel can you s

In [31]:
times2 = pd.DataFrame(times2)
times2.describe()

,duration
count,10.000000
mean,0.669434
std,0.215863
min,0.403813
25%,0.608974
50%,0.609961
75%,0.613053
max,1.224218


**Takeaways**

At least in this tiny benchmark, openai's model 1 is ~10x faster than banana's model. Both of these are ~6B params.